"Термин -- это слово, из которого изъята жизнь" (С.Я. Маршак)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
from pymystem3 import Mystem
mystem = Mystem()

import nltk
from nltk.util import ngrams

import spacy
spacy.cli.download("ru_core_news_lg")
nlp = spacy.load('ru_core_news_lg')

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
df = pd.read_json("/content/drive/MyDrive/ML_Data/train_t1_v1.jsonl", lines=True)
df.head()

,id,text,keywords,label
0,2000-1,ABBYY Retrieval & Morphology Engine\nВ сообщен...,,"[[0, 35], [6, 15], [18, 35], [29, 35], [69, 88..."
1,2000-2,Речевые формулы в диалоге\nПредложенная класси...,,"[[0, 15], [18, 25], [74, 99], [134, 140], [175..."
2,2000-4,Географические названия и полнотекстовые докум...,,"[[0, 23], [26, 50], [54, 68], [169, 190], [181..."
3,2000-5,Методы автоматического построения специализиро...,,"[[7, 63], [34, 63], [54, 63], [92, 128], [119,..."
4,2000-6,Закономерности построения дискурсивной последо...,,"[[26, 57], [62, 76], [251, 265], [266, 280], [..."


In [5]:
# лемматизация терминов и текстов

def lemmatize(text):
  return ' '.join([i for i in mystem.lemmatize(text) if i[0].isalpha()])

In [6]:
n_for_ngrams = 4

In [7]:
def get_terms(x):

  terms = []
  for i in x['label']:
    start = i[0]
    end = i[1]
    term = x['text'][start:end]
    terms.append(term)

  return sorted(list(set([lemmatize(term) for term in terms])))

In [8]:
df['terms'] = df.apply(get_terms, axis=1) # добавляем стоблец с лемматизироавнными уникальными терминами

In [9]:
df['lem_text'] = df['text'].apply(lemmatize) # добавляем стоблец с лемматизироавнными текстами

In [10]:
df = df[['text', 'lem_text', 'terms']]

In [11]:
df.head()

,text,lem_text,terms
0,ABBYY Retrieval & Morphology Engine\nВ сообщен...,ABBYY Retrieval Morphology Engine в сообщение ...,"[ABBYY Retrieval Morphology Engine, Engine, Mo..."
1,Речевые формулы в диалоге\nПредложенная класси...,речевой формула в диалог предлагать классифика...,"[диалог, идиом, идиома, иллокутивный, коммента..."
2,Географические названия и полнотекстовые докум...,географический название и полнотекстовый докум...,"[генерация, географический название, геологиче..."
3,Методы автоматического построения специализиро...,метод автоматический построение специализирова...,"[SwissProt, автоматический построение специали..."
4,Закономерности построения дискурсивной последо...,закономерность построение дискурсивный последо...,"[ДП, ДС, высказывание, дискурсивный последоват..."


In [12]:
def get_ngrams(lemmatized_text, n): # получаем для каждого предложения все возможные n-граммы от n = 1 до n = n

  lemmas_lst = lemmatized_text.split()

  n_grams = []
  for n in range(1, n + 1):
    n_grams_item = [list(i) for i in list(ngrams((lemmas_lst), n))]
    for i in n_grams_item:
      n_grams.append(i)

  return [' '.join(i) for i in n_grams]

In [13]:
n_grams_plus_sents = []


for i in range(len(df['lem_text'])):
  terms = df['terms'][i]
  n_grams = get_ngrams(df['lem_text'][i], n_for_ngrams)
  for ngram in n_grams:
    single_list = []
    single_list.append(ngram)
    single_list.append(df['text'][i])
    single_list.append(int(ngram in terms))

    n_grams_plus_sents.append(single_list)

In [14]:
# создаем новый датафрейм с n-граммами, для каждой из которых указано
# предложение-источник, а также лейбл класса(является ли данная n-грамма термином)

new_df = pd.DataFrame(n_grams_plus_sents, columns=['ngram', 'sent', 'label'])
new_df

,ngram,sent,label
0,ABBYY,ABBYY Retrieval & Morphology Engine\nВ сообщен...,0
1,Retrieval,ABBYY Retrieval & Morphology Engine\nВ сообщен...,1
2,Morphology,ABBYY Retrieval & Morphology Engine\nВ сообщен...,0
3,Engine,ABBYY Retrieval & Morphology Engine\nВ сообщен...,1
4,в,ABBYY Retrieval & Morphology Engine\nВ сообщен...,0
...,...,...,...
261039,также оцениваться качество работа,Сопоставительный анализ морфопарсеров корейско...,0
261040,оцениваться качество работа парсер,Сопоставительный анализ морфопарсеров корейско...,0
261041,качество работа парсер UDPipe,Сопоставительный анализ морфопарсеров корейско...,0
261042,работа парсер UDPipe и,Сопоставительный анализ морфопарсеров корейско...,0


In [ ]:
# расставляем частеречные теги

In [18]:
def get_pos_list(x):

  ngram = x['ngram']
  doc = nlp(ngram)

  pos_list = []
  for token in doc:
    pos_list.append(token.pos_)
  for i in range(n_for_ngrams - len(pos_list)):
    pos_list.append(0)

  return pos_list

In [19]:
new_df['pos_tags'] = new_df.apply(get_pos_list, axis=1)

In [20]:
new_df

,ngram,sent,label,pos_tags
0,ABBYY,ABBYY Retrieval & Morphology Engine\nВ сообщен...,0,"[PROPN, 0, 0, 0]"
1,Retrieval,ABBYY Retrieval & Morphology Engine\nВ сообщен...,1,"[PROPN, 0, 0, 0]"
2,Morphology,ABBYY Retrieval & Morphology Engine\nВ сообщен...,0,"[PROPN, 0, 0, 0]"
3,Engine,ABBYY Retrieval & Morphology Engine\nВ сообщен...,1,"[PROPN, 0, 0, 0]"
4,в,ABBYY Retrieval & Morphology Engine\nВ сообщен...,0,"[ADP, 0, 0, 0]"
...,...,...,...,...
261039,также оцениваться качество работа,Сопоставительный анализ морфопарсеров корейско...,0,"[ADV, VERB, NOUN, NOUN]"
261040,оцениваться качество работа парсер,Сопоставительный анализ морфопарсеров корейско...,0,"[VERB, NOUN, NOUN, VERB]"
261041,качество работа парсер UDPipe,Сопоставительный анализ морфопарсеров корейско...,0,"[NOUN, NOUN, VERB, PROPN]"
261042,работа парсер UDPipe и,Сопоставительный анализ морфопарсеров корейско...,0,"[NOUN, VERB, PROPN, CCONJ]"


In [21]:
new_df.to_csv('/content/drive/MyDrive/ML_Data/prerpocessed_for_embeddings.csv', sep=';', encoding='utf-8')